In [1]:
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
import numpy as np
from pathlib import Path

In [2]:
batch_size = 64

In [3]:
train_gen = ImageDataGenerator(
                               validation_split=0.15,
                               preprocessing_function=preprocess_input)

test_gen = ImageDataGenerator(
                             preprocessing_function=preprocess_input)

In [4]:
training_set = train_gen.flow_from_directory(
    directory="jpegs/",
    target_size=(224,224),
    class_mode='categorical',
    subset='training',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

validation_set = train_gen.flow_from_directory(
    directory="jpegs/",
    target_size=(224,224),
    class_mode='categorical',
    subset='validation',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

test_set = test_gen.flow_from_directory(
    directory="jpegs_validation/",
    target_size=(224,224),
    class_mode=None,
    batch_size=1,
    shuffle=False,
    seed=42
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [5]:
training_set.class_indices

{}

In [6]:
for _ in range(5):
    img, label = training_set.next()
    for i in range(batch_size):
        #print(label[i])
        print(img.shape)
        plt.imshow(img[i])
        plt.title(''.join([k for k,v in training_set.class_indices.items() if v == int(np.where(np.isclose(label[i], 1.))[0])]))
        plt.show()
    break

(0, 224, 224, 3)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
# include_top =False to remove the Fully-Connected layer which generates 1000 different output labels whereas
# our Target dataset has only 5 classes for prediction
vggmodel = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
for layer in (vggmodel.layers):
    layer.trainable = False

In [9]:
x = Flatten()(vggmodel.output)

x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)


In [10]:
predictions_output = Dense(5, activation='softmax', kernel_initializer='random_uniform',
                    bias_initializer='random_uniform', bias_regularizer=regularizers.l2(0.01), name='predictions')(x)


In [11]:
model_final = Model(inputs = vggmodel.input, outputs = predictions_output)

In [12]:
training_size = 1424
validation_size = 249
steps_per_epoch = math.ceil(training_size / batch_size)
validation_steps = math.ceil(validation_size / batch_size)

In [13]:
# compilation 1
rms = optimizers.RMSprop(lr=0.0001, decay=1e-4)
model_final.compile(loss="categorical_crossentropy", optimizer = rms, metrics=["accuracy"])

In [ ]:
model_final.summary()

In [ ]:
earlystop1 = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

In [ ]:
hist1 = model_final.fit_generator(
    training_set,
    steps_per_epoch=steps_per_epoch,
    epochs=4,
    validation_data = test_set,
    validation_steps=validation_steps,
    callbacks=[earlystop1],
    workers=10,
    shuffle=True
)

In [ ]:
plt.plot(hist1.history["accuracy"])
plt.plot(hist1.history['val_accuracy'])
plt.plot(hist1.history['loss'])
plt.plot(hist1.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [14]:
model_final.save("checkpoint/object_vgg16model.h5")

In [ ]:
# Generate predictions output
model_final.load_weights('checkpoint/object_vgg16model.h5')


In [ ]:
true_classes = test_set.classes

In [ ]:
vgg_preds = model_final.predict(test_set)

In [ ]:
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)

In [ ]:
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))